In [1]:
# Import dependencies
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import resnet18, ResNet18_Weights #pretrained model

In [2]:
# Define the training and validation transforms for the data, add data aug to increase dateset sample
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  # Flips images vertically
    transforms.RandomRotation(30),  # Increase rotation range. I've tried 10 and 20
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),  # Increase color jitter
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),  # Wider scale for random cropping
    transforms.RandomGrayscale(p=0.2),  # Randomly convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [4]:
# Load the entire dataset
data_dir = '/content/drive/MyDrive/AI Bootcamp/animals/animals'
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)


In [5]:
# Define Animal Classes
animal_classes = ['cat', 'cow', 'coyote', 'deer', 'dog', 'donkey', 'fox', 'horse', 'owl', 'pig', 'possum', 'raccoon', 'sheep', 'wolf']

# Define predator mapping (0: nonpredator, 1: predator, 2: both)
predator_mapping = {'cat': 2, 'cow': 0, 'coyote': 1, 'deer': 2, 'dog': 2, 'donkey': 0, 'fox': 1, 'horse': 0, 'owl': 1, 'pig': 0, 'possum': 1,
                    'raccoon': 1, 'sheep': 0, 'wolf': 1}

predator_classes = ['nonpredator', 'predator', 'both']

In [6]:
# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

In [7]:
# Apply validation transforms to val_dataset
val_dataset.dataset.transform = val_transforms

In [8]:
# Create DataLoaders
batch_size = 64  # Adjust batch size. Tried 32 and 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2) # Changed num_workers from 4 to 2 based on Colab warning
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [9]:
# Create model
class AnimalClassifier(nn.Module):
    def __init__(self, num_animal_classes=14):
        super(AnimalClassifier, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Pretrained model w/ weights
        self.model.fc = nn.Linear(self.model.fc.in_features, num_animal_classes)

    def forward(self, x):
        return self.model(x)

In [10]:
# Load the model
model = AnimalClassifier()

In [11]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Use label smoothing
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)  # Use SGD with weight decay

In [12]:
# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

In [14]:
# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
            total_preds += len(labels)

    epoch_loss = running_loss / len(val_loader.dataset)
    accuracy = correct_preds.double() / total_preds
    return epoch_loss, accuracy

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


AnimalClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [15]:
# Received a os.fork() is incompatible with multithreaded code
# Chat GPT suggested the following fix
import multiprocessing as mp

# Set the start method to 'spawn' at the beginning of your script
mp.set_start_method('spawn', force=True)



# Train the model with early stopping
num_epochs = 50
patience = 5
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Accuracy: {val_accuracy:.4f}")

    scheduler.step()

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs")
        break


Epoch 1/50, Train Loss: 2.3143, Val Loss: 1.2562, Val Accuracy: 0.7619
Epoch 2/50, Train Loss: 0.9390, Val Loss: 1.0513, Val Accuracy: 0.8333
Epoch 3/50, Train Loss: 0.7663, Val Loss: 1.1051, Val Accuracy: 0.8095
Epoch 4/50, Train Loss: 0.7047, Val Loss: 1.0530, Val Accuracy: 0.8333
Epoch 5/50, Train Loss: 0.6644, Val Loss: 1.0073, Val Accuracy: 0.8175
Epoch 6/50, Train Loss: 0.6496, Val Loss: 1.0440, Val Accuracy: 0.8175
Epoch 7/50, Train Loss: 0.6246, Val Loss: 0.9853, Val Accuracy: 0.8413
Epoch 8/50, Train Loss: 0.6087, Val Loss: 0.9698, Val Accuracy: 0.8333
Epoch 9/50, Train Loss: 0.6034, Val Loss: 0.9682, Val Accuracy: 0.8492
Epoch 10/50, Train Loss: 0.5991, Val Loss: 0.9695, Val Accuracy: 0.8492
Epoch 11/50, Train Loss: 0.5960, Val Loss: 0.9724, Val Accuracy: 0.8413
Epoch 12/50, Train Loss: 0.5930, Val Loss: 0.9700, Val Accuracy: 0.8413
Epoch 13/50, Train Loss: 0.5936, Val Loss: 0.9717, Val Accuracy: 0.8571
Epoch 14/50, Train Loss: 0.5921, Val Loss: 0.9706, Val Accuracy: 0.8492
E

In [23]:
torch.save(model, 'animal_classifier_model.pt')

In [16]:
# Function to classify an image and return the animal and predator class
def classify_image(image):
    model.eval()
    with torch.no_grad():
        image = val_transforms(image).unsqueeze(0).to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        animal_class = animal_classes[predicted.item()]
        predator_class = predator_classes[predator_mapping[animal_class]]
        return animal_class, predator_class

In [21]:
import gradio as gr
from PIL import Image

# Set up the Gradio interface
def predict(image):
    animal_class, predator_class = classify_image(image)
    return f"Animal: {animal_class}, Category: {predator_class}"

interface = gr.Interface(fn=predict, inputs=gr.Image(type="pil"), outputs=gr.Text(), title="Animal Classifier")
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ace4bff556d9a7f43c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
